In [1]:
import geopandas as gpd
import pandas as pd
import sqlite3
import os

# Importando os bancos de dados

In [2]:
conn = sqlite3.connect(os.path.join('pib.db'))
query= "SELECT * FROM pib"
pib_df = pd.read_sql(query,conn)
conn.close()
pib=pib_df.copy()
pib_df.head(1)

,index,id,gid,UF,nome,Censo,PIB,Pop_est_2009,PIB_percapita,legenda,classe,geometry
0,0,vw_pib_percapita.fid-2e5b397f_17cf1b12db8_-5754,215,BAHIA,Tremedal,2010,57883.992,18433.0,3140.24,- 9639.65,1,"MULTIPOLYGON (((-41.45880331 -14.78141546, -41..."


In [3]:
conn = sqlite3.connect(os.path.join('racial.db'))
query= "SELECT * FROM racial"
racial_df = pd.read_sql(query,conn)
conn.close()
racial=racial_df.copy()
racial_df.head(1)

,index,id,gid,UF,nome,Censo,POP_TOT,Pessoas_Pretas,PER_Pessoas_Pretas,descricao,legenda,classe,geometry
0,0,vw_per_pessoas_pretas.fid-60efe8e1_17d582643c1...,215,BAHIA,Tremedal,2010,17029.0,827,4.856421,Percentual de pessoas residentes de cor ou raç...,"4,22% - 7,79%",2,"MULTIPOLYGON (((-41.45880331 -14.78141546, -41..."


### Transformando os DataFrames para o tipo GeoDataFrame

In [4]:
pib['geometry']=gpd.GeoSeries.from_wkt(pib_df['geometry'])
pib = gpd.GeoDataFrame(pib, geometry='geometry')
pib.head(1)

AttributeError: type object 'GeoSeries' has no attribute 'from_wkt'

In [ ]:
racial['geometry']=gpd.GeoSeries.from_wkt(racial['geometry'])
racial = gpd.GeoDataFrame(racial, geometry='geometry')
racial.head(1)

## Tratando os dados

In [ ]:
pib = pib.sort_values('nome')
pib['nome'] = pib['nome'].str.title()
pib.head(2)

In [ ]:
racial = racial.sort_values('nome')
racial['nome'] = racial['nome'].str.title()
racial.head(2)

### Unindo os Dataframes sobre o PIB e a população negra

In [ ]:
racial_dfn = racial_df.drop(racial_df.columns[[0,1,3,4,5,9,12]], axis=1) #retirando colunas iguais
merge = pd.merge(pib_df, racial_dfn, how='outer', on='gid')

Transformando em GeoDataFrame

In [ ]:
merge['geometry']=gpd.GeoSeries.from_wkt(merge['geometry'])
merge = gpd.GeoDataFrame(merge, geometry='geometry')
merge.head(1)

## Analisando os dados

In [ ]:
merge['PIB_percapita'] = merge['PIB_percapita'].astype(float)
merge['Pop_est_2009'] = merge['Pop_est_2009'].astype(float)
merge['PER_Pessoas_Pretas'] = merge['PER_Pessoas_Pretas'].astype(float)

In [ ]:
merge[['PIB', 'Pop_est_2009', 'PIB_percapita', 'PER_Pessoas_Pretas']].agg(['mean', 'median'])

## Plotando os dados

In [ ]:
merge.plot(color='green', edgecolor='None', figsize=(15,8))